# 🔐 Authentication & Security Overview

This notebook covers essential backend security concepts: authentication vs authorization, token-based auth with JWT, OAuth 2.0 flows, password hashing, and security headers.

---

## 1. Authentication vs Authorization

| Aspect | Authentication (AuthN) | Authorization (AuthZ) |
|--------|------------------------|----------------------|
| **Question** | "Who are you?" | "What can you do?" |
| **Purpose** | Verify identity | Grant permissions |
| **Happens** | First | After authentication |
| **Methods** | Password, MFA, Biometrics, SSO | RBAC, ABAC, ACLs, Policies |
| **Failure** | 401 Unauthorized | 403 Forbidden |

**Key Insight**: Authentication establishes *identity*, authorization enforces *access control*.

## 2. JWT (JSON Web Tokens)

JWTs are self-contained tokens for stateless authentication.

```
┌─────────────────────────────────────────────────────────────────────┐
│                         JWT Structure                                │
├─────────────────┬─────────────────────┬─────────────────────────────┤
│     HEADER      │      PAYLOAD        │         SIGNATURE           │
│   (Algorithm)   │   (Claims/Data)     │   (Verification)            │
├─────────────────┼─────────────────────┼─────────────────────────────┤
│ {               │ {                   │ HMACSHA256(                 │
│   "alg":"HS256",│   "sub": "user123", │   base64(header) + "." +    │
│   "typ":"JWT"   │   "exp": 1735689600,│   base64(payload),          │
│ }               │   "role": "admin"   │   secret                    │
│                 │ }                   │ )                           │
└─────────────────┴─────────────────────┴─────────────────────────────┘
         ↓                   ↓                        ↓
    Base64Url          Base64Url                 Base64Url
         └────────────── . ──────────────────── . ────┘
                    Final Token: xxxxx.yyyyy.zzzzz
```

**Standard Claims**: `iss` (issuer), `sub` (subject), `exp` (expiration), `iat` (issued at), `aud` (audience)

In [ ]:
# JWT Example with PyJWT
import jwt
from datetime import datetime, timedelta

SECRET_KEY = "your-secret-key-keep-it-safe"

# Create a JWT token
def create_token(user_id: str, role: str) -> str:
    payload = {
        "sub": user_id,
        "role": role,
        "iat": datetime.utcnow(),
        "exp": datetime.utcnow() + timedelta(hours=1)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")

# Verify and decode a JWT token
def verify_token(token: str) -> dict:
    try:
        return jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        raise ValueError("Token has expired")
    except jwt.InvalidTokenError:
        raise ValueError("Invalid token")

# Demo
token = create_token("user_123", "admin")
print(f"Token: {token[:50]}...")
print(f"Decoded: {verify_token(token)}")

## 3. OAuth 2.0 Flows

OAuth 2.0 is a delegation protocol — it allows apps to access resources on behalf of users without sharing credentials.

### Authorization Code Flow (Most Secure - for server-side apps)

```
┌──────────┐                              ┌───────────────┐                              ┌──────────────┐
│   User   │                              │  Your App     │                              │ Auth Server  │
│ (Browser)│                              │  (Backend)    │                              │ (Google/etc) │
└────┬─────┘                              └───────┬───────┘                              └──────┬───────┘
     │                                            │                                             │
     │ 1. Click "Login with Google"               │                                             │
     │ ──────────────────────────────────────────>│                                             │
     │                                            │                                             │
     │ 2. Redirect to Auth Server                 │                                             │
     │ <──────────────────────────────────────────│                                             │
     │                                            │                                             │
     │ 3. User logs in & grants permission        │                                             │
     │ ─────────────────────────────────────────────────────────────────────────────────────────>
     │                                            │                                             │
     │ 4. Redirect back with Authorization Code   │                                             │
     │ <─────────────────────────────────────────────────────────────────────────────────────────
     │                                            │                                             │
     │                                            │ 5. Exchange code for tokens (server-to-server)
     │                                            │ ───────────────────────────────────────────>│
     │                                            │                                             │
     │                                            │ 6. Access Token + Refresh Token             │
     │                                            │ <───────────────────────────────────────────│
     │                                            │                                             │
     │ 7. User is authenticated                   │                                             │
     │ <──────────────────────────────────────────│                                             │
```

### OAuth 2.0 Flow Comparison

| Flow | Use Case | Token Exposure |
|------|----------|----------------|
| **Authorization Code** | Server-side web apps | Tokens stay on server ✅ |
| **Authorization Code + PKCE** | SPAs, Mobile apps | Code verifier protects exchange ✅ |
| **Client Credentials** | Machine-to-machine | No user involved |
| **Implicit** ⚠️ | Legacy SPAs | Token in URL (deprecated) ❌ |

## 4. Password Hashing with bcrypt

**Never store passwords in plain text!** Use slow, salted hashing algorithms.

```
┌─────────────────────────────────────────────────────────────────┐
│                    Password Hashing Process                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   "mypassword"  ──┬──> Salt (random) ──> bcrypt() ──> Hash      │
│                   │                                              │
│                   └──> Cost Factor (work factor, e.g., 12)       │
│                                                                  │
│   Stored: $2b$12$LQv3c1yqBWVHxkd0LHAkCOYz6TtxMQJqhN8/X4.G...    │
│            ↑   ↑   ↑                                            │
│          algo cost  salt+hash                                   │
└─────────────────────────────────────────────────────────────────┘
```

**Why bcrypt?**
- Slow by design (prevents brute force)
- Built-in salt (prevents rainbow tables)
- Configurable cost factor (future-proof)

In [ ]:
# Password Hashing with bcrypt
import bcrypt

def hash_password(password: str) -> bytes:
    """Hash a password with auto-generated salt."""
    salt = bcrypt.gensalt(rounds=12)  # Cost factor of 12
    return bcrypt.hashpw(password.encode('utf-8'), salt)

def verify_password(password: str, hashed: bytes) -> bool:
    """Verify a password against its hash."""
    return bcrypt.checkpw(password.encode('utf-8'), hashed)

# Demo
password = "SecureP@ssw0rd!"
hashed = hash_password(password)

print(f"Original:  {password}")
print(f"Hashed:    {hashed.decode()}")
print(f"Verify OK: {verify_password(password, hashed)}")
print(f"Wrong pwd: {verify_password('wrongpassword', hashed)}")

## 5. Essential Security Headers

Security headers protect against common web vulnerabilities. Configure these at the web server or application level.

| Header | Purpose | Example Value |
|--------|---------|---------------|
| `Strict-Transport-Security` | Force HTTPS | `max-age=31536000; includeSubDomains` |
| `Content-Security-Policy` | Prevent XSS, injection | `default-src 'self'; script-src 'self'` |
| `X-Content-Type-Options` | Prevent MIME sniffing | `nosniff` |
| `X-Frame-Options` | Prevent clickjacking | `DENY` or `SAMEORIGIN` |
| `X-XSS-Protection` | Legacy XSS filter | `1; mode=block` |
| `Referrer-Policy` | Control referrer info | `strict-origin-when-cross-origin` |
| `Permissions-Policy` | Restrict browser features | `geolocation=(), camera=()` |

In [ ]:
# Security Headers Middleware Example (Flask/FastAPI style)

SECURITY_HEADERS = {
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains; preload",
    "Content-Security-Policy": "default-src 'self'; script-src 'self'; style-src 'self' 'unsafe-inline'",
    "X-Content-Type-Options": "nosniff",
    "X-Frame-Options": "DENY",
    "X-XSS-Protection": "1; mode=block",
    "Referrer-Policy": "strict-origin-when-cross-origin",
    "Permissions-Policy": "geolocation=(), camera=(), microphone=()"
}

# Flask example
def add_security_headers(response):
    for header, value in SECURITY_HEADERS.items():
        response.headers[header] = value
    return response

# FastAPI example
from fastapi import FastAPI
from starlette.middleware.base import BaseHTTPMiddleware

class SecurityHeadersMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        response = await call_next(request)
        for header, value in SECURITY_HEADERS.items():
            response.headers[header] = value
        return response

print("Security headers configured:")
for h, v in SECURITY_HEADERS.items():
    print(f"  {h}: {v[:40]}{'...' if len(v) > 40 else ''}")

## 6. Quick Reference: Security Checklist

### ✅ Authentication Best Practices
- [ ] Use HTTPS everywhere (HSTS header)
- [ ] Hash passwords with bcrypt/Argon2 (cost factor ≥ 12)
- [ ] Implement rate limiting on login endpoints
- [ ] Use secure session cookies (`HttpOnly`, `Secure`, `SameSite=Strict`)
- [ ] Implement MFA for sensitive operations

### ✅ Token Security
- [ ] Short JWT expiration (15-60 min for access tokens)
- [ ] Use refresh token rotation
- [ ] Store tokens securely (HttpOnly cookies > localStorage)
- [ ] Validate `iss`, `aud`, `exp` claims

### ✅ OAuth 2.0
- [ ] Always use PKCE for public clients
- [ ] Validate redirect URIs strictly
- [ ] Use state parameter to prevent CSRF

---
📚 **Further Reading**: OWASP Authentication Cheatsheet, RFC 7519 (JWT), RFC 6749 (OAuth 2.0)